# Load Required Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import DBSCAN
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import gc
import pickle

# Load

In [2]:
Folder = "E:\\datasets\\car\\"

In [3]:
Sample = pd.read_parquet(f"{Folder}SampleEncoded.parq")
Sample

,mileage,engine_displacement,engine_power,price_eur,maker_alfa-romeo,maker_aston-martin,maker_audi,maker_bentley,maker_bmw,maker_chevrolet,...,seat_count_4.0,seat_count_5.0,seat_count_6.0,seat_count_7.0,seat_count_8.0,seat_count_9.0,seat_count_nan,fuel_type_diesel,fuel_type_gasoline,fuel_type_nan
0,0.777224,0.218001,0.111079,-0.228444,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
1,0.683206,0.218001,-0.411832,-0.363319,False,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
2,0.110900,0.210865,-0.316757,-0.111166,False,False,False,False,True,False,...,False,True,False,False,False,False,False,True,False,False
3,0.289515,-0.923750,-0.839668,-0.832452,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,0.500893,-0.923750,-0.839668,-0.850044,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False


In [4]:
ExclusionColumns = [
    # "model_nan",
    # "maker_nan",
    # "mileage",
    # "engine_displacement",
    # "engine_power",
]

EncodedData = pd.read_parquet(f"{Folder}Encoded.parq")

for c in ExclusionColumns:
    if EncodedData.dtypes[c] == bool:
        EncodedData = EncodedData[EncodedData[c] == False]
    else:
        EncodedData = EncodedData[EncodedData[c].notnull()]

EncodedData = EncodedData.reset_index(drop=True)

EncodedData = EncodedData.fillna(0)

if "X" in globals():
    del globals()["X"]

if "y" in globals():
    del globals()["y"]

X = EncodedData[[x for x in EncodedData.columns if x != "price_eur"]]
y = EncodedData[["price_eur"]]

if "EncodedData" in globals():
    del globals()["EncodedData"]

gc.collect()

0

# Split into Training & Testing

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.40, random_state=1991
)

if "X" in globals():
    del globals()["X"]

if "y" in globals():
    del globals()["y"]

gc.collect()

0

# K Neighbors Regressor

In [6]:
kn = KNeighborsRegressor(25, weights="distance")
kn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=25, weights='distance')

In [7]:
with open("PriceScaler.obj", "rb") as PS_File:
    PriceScaler = pickle.load(PS_File)

In [23]:
Start = 150000
Number = 500
y_pred = kn.predict(X_test.iloc[Start:Start+Number])

In [24]:
Comparison = pd.concat(
    [pd.DataFrame(y_pred), y_test.iloc[Start:Start+Number].reset_index(drop=True)],
    axis=1,
)
Comparison = pd.DataFrame(
    PriceScaler.inverse_transform(Comparison), columns=["Predicted", "Price"]
)
print(len(Comparison))
display(Comparison.head())
r2_score(Comparison["Price"], Comparison["Predicted"])

500


,Predicted,Price
0,9824.338731,8400.0
1,15776.980000,15861.1
2,3325.083557,3491.3
3,11004.673020,10990.0
4,10484.017627,12990.0


0.9411952607275916

# DBScan

In [ ]:
db = DBSCAN(min_samples=100)
db.fit(X)

# Decision Tree Regressor

In [ ]:
reg = DecisionTreeRegressor().fit(X, y)

In [ ]:
y_pred = reg.predict(X)

In [ ]:
r2_score(y_pred, y)

In [ ]:
# dummies["Y"] = y
# dummies["Pred"] = y_pred
# dummies["Diff"] = (dummies["Y"] - dummies["Pred"]).abs()
# gc.collect()
# Data = pd.read_parquet("clean_df.parq")
# Data["Pred"] = dummies["Pred"]
# Data["Diff"] = dummies["Diff"]
# Data = Data.sort_values(by="Diff", ascending=False)
# Data.to_parquet("PredictionDifferences.parq")
# Data[(Data["maker"] == "volkswagen") & (Data["model"] == "tiguan")]["price_eur"].hist()